In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import json
import re


## 1. Carga y visualización de datasets

In [2]:
# carga raw_reviews.parquet
df_reviews= pd.read_parquet(r'C:\Users\Lenovo\Desktop\Proyecto Individual\Datasets\0. Data\raw_reviews.parquet')

In [3]:
# carga raw_items.parquet
df_items= pd.read_parquet(r'C:\Users\Lenovo\Desktop\Proyecto Individual\Datasets\0. Data\raw_items.parquet')

In [4]:
# carga raw_games.parquet
df_games = pd.read_json(r'C:\Users\Lenovo\Desktop\Proyecto Individual\Datasets\0. Data\raw_games.json', lines=True)

In [5]:
# Echamos un vistazo a df_reviews
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'helpful': 'No ratings yet', 'i..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'helpful': '15 of 20 people (75..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'helpful': 'No ratings yet', 'i..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'helpful': '2 of 2 people (100%..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [6]:
# Revisamos la visualización de df_items
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [7]:
# por último, revisamos la visualización de df_steam
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


## 1. ETL df_reviews

In [8]:
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'helpful': 'No ratings yet', 'i..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'helpful': '15 of 20 people (75..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'helpful': 'No ratings yet', 'i..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'helpful': '2 of 2 people (100%..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


*Se crea copia para realizar las transformaciones*

In [9]:
df_reviews_copy = df_reviews.copy() # Hacemos una copia del DataFrame para hacer modificaciones del ETL

In [10]:
#revisamos los tipos de datos

df_reviews_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


Desanidado de reviews

In [11]:
# Trabajamos con extraer los datos de la columna reviews, ya que viene anidada

df_reviews_copy['reviews'][25798][0:]

array([{'funny': '1 person found this review funny', 'helpful': '1 of 2 people (50%) found this review helpful', 'item_id': '273110', 'last_edited': '', 'posted': 'Posted July 3.', 'recommend': True, 'review': 'had so much fun plaing this and collecting resources xD we won on my first try and killed final boss!'},
       {'funny': '', 'helpful': 'No ratings yet', 'item_id': '730', 'last_edited': '', 'posted': 'Posted July 20.', 'recommend': True, 'review': ':D'},
       {'funny': '', 'helpful': 'No ratings yet', 'item_id': '440', 'last_edited': '', 'posted': 'Posted July 2.', 'recommend': True, 'review': 'so much fun :D'}],
      dtype=object)

Al revisar estos datos, en conjunto con el diccionario de datos disponible, se puede entender que este datasets guarda de forma anidada todos los reviews realizados por éste para diferentes ítems  
(videojuegos), que es información que necesitamos desagregar para crear las funciones solicitadas. 
Se procede a trabajar en este desanidado.

In [12]:
# Crea función convert_to_list para verificar si los datos en 'reviews' son listas o cadenas y procesar
def convert_to_list(item):
    if isinstance(item, str):  # Si el dato es una cadena, intentamos convertirlo
        try:
            item = json.loads(item.replace("'", '"'))  # Reemplazar comillas simples y cargar JSON
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
    return item  # Si ya es una lista, se devuelve tal cual

In [13]:

# Aplicar la función solo si es necesario
df_reviews_copy['reviews'] = df_reviews_copy['reviews'].apply(convert_to_list)

# Usar explode() para expandir los comentarios en varias filas
df_reviews_copy_expanded = df_reviews_copy.explode('reviews').reset_index(drop=True)

# Desanidar el contenido de la columna 'reviews' en columnas separadas
df_reviews_copy_final = pd.json_normalize(df_reviews_copy_expanded['reviews'])

# Unir los datos originales con la información desanidada
df_reviews_final = pd.concat([df_reviews_copy_expanded.drop(columns=['reviews']), df_reviews_copy_final], axis=1)



In [14]:
df_reviews_final.head()

,user_id,user_url,funny,helpful,item_id,last_edited,posted,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,No ratings yet,1250,,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,No ratings yet,22200,,"Posted July 15, 2011.",True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,No ratings yet,43110,,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,15 of 20 people (75%) found this review helpful,251610,,"Posted June 24, 2014.",True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,0 of 1 people (0%) found this review helpful,227300,,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...


*En algunas columnas se aprecian valores vacíos. Convertimos todos los valores vacíos del dataframe en valores nulos para poder analizarlos posteriormente*

In [15]:
# Reemplazar todos los valores vacíos ('') en el df_reviews_final por NaN
df_reviews_final.replace('', np.nan, inplace=True)

*Se eliminan las filas duplicadas del dataframe*

In [16]:
# Se eliminan posibles duplicados
df_reviews_final.drop_duplicates(inplace=True)

In [17]:
df_reviews_final.isnull().sum()

user_id            0
user_url           0
funny          50449
helpful           28
item_id           28
last_edited    52422
posted            28
recommend         28
review            58
dtype: int64

In [18]:
df_reviews_final.sample(5)

,user_id,user_url,funny,helpful,item_id,last_edited,posted,recommend,review
54698,bluemangreg,http://steamcommunity.com/id/bluemangreg,NaN,2 of 2 people (100%) found this review helpful,4000,NaN,"Posted June 24, 2014.",True,FUN
21483,tsunamitad,http://steamcommunity.com/id/tsunamitad,461 people found this review funny,668 of 722 people (93%) found this review helpful,72200,NaN,"Posted May 18, 2015.",True,What can I say?1 hour of gameplay to explore t...
27013,checkm80,http://steamcommunity.com/id/checkm80,NaN,1 of 1 people (100%) found this review helpful,105450,NaN,Posted June 30.,True,One of the best RTS games I've ever played. Go...
16786,piedude,http://steamcommunity.com/id/piedude,NaN,2 of 2 people (100%) found this review helpful,255710,NaN,"Posted March 15, 2015.",True,This is one game you dont want to ignore! like...
9786,76561198219780653,http://steamcommunity.com/profiles/76561198219...,NaN,3 of 6 people (50%) found this review helpful,310380,NaN,Posted July 22.,True,Great!


In [19]:
df_reviews_final[df_reviews_final['review'].isnull()].sample(5)

,user_id,user_url,funny,helpful,item_id,last_edited,posted,recommend,review
43114,STEAM0082987612,http://steamcommunity.com/id/STEAM0082987612,NaN,No ratings yet,224260,NaN,"Posted December 20, 2015.",False,NaN
31955,76561198110176420,http://steamcommunity.com/profiles/76561198110...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45409,viihdavanzo,http://steamcommunity.com/id/viihdavanzo,NaN,No ratings yet,730,NaN,"Posted September 9, 2015.",True,NaN
32604,76561198072577997,http://steamcommunity.com/profiles/76561198072...,NaN,1 of 1 people (100%) found this review helpful,570,NaN,"Posted December 20, 2014.",True,NaN
20223,76561198079342142,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Hallazgos: 
- df_reviews_final tiene 28 valores nulos en las columnas 'user_url', 'funny', 'last_edited', 'helpful', 'posted', 'item_id' , 'recommend' y 'review'.
- Si el usuario tiene um item asociado, al parecer está obligado a interactuar redactando algún 'review' o recomendando o no el juego ('recommend'), y es cuando se registra la fecha en 'posted'
- Si no existe información en estas columnas 'funny', 'last_edited', 'helpful', 'posted', 'item_id' , 'recommend' y 'review', al parecer se trataría de usuarios registrados que no tienen un item asociado, se podrían eliminar.
- Existen columnas que podemos eliminar: 'user_url', 'funny', 'last_edited', 'helpful'
- Se requieren hacer ajustes en los tipos de datos: 'item_id' y 'recommend'

Eliminación de columnas

In [20]:
# se eliminan columnas que no necesito: user_url, funny, last_edited, helpful 

columnas=['user_url', 'funny', 'last_edited', 'helpful']
df_reviews_final.drop(columnas,axis='columns', inplace=True)

In [21]:
df_reviews_final

,user_id,item_id,posted,recommend,review
0,76561197970982479,1250,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...
1,76561197970982479,22200,"Posted July 15, 2011.",True,It's unique and worth a playthrough.
2,76561197970982479,43110,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,251610,"Posted June 24, 2014.",True,I know what you think when you see this title ...
4,js41637,227300,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...
59328,76561198312638244,70,Posted July 10.,True,a must have classic from steam definitely wort...
59329,76561198312638244,362890,Posted July 8.,True,this game is a perfect remake of the original ...
59330,LydiaMorley,273110,Posted July 3.,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,730,Posted July 20.,True,:D


Tratamiento de valores nulos

In [22]:
# Se revisan los valores nulos en este dataframe
df_reviews_final.isnull().sum()

user_id       0
item_id      28
posted       28
recommend    28
review       58
dtype: int64

In [23]:
# se revisan los nulos del df_reviews_final
nulos_reviews = df_reviews_final[df_reviews_final[['item_id', 'recommend', 'review']].isna().any(axis=1)]
print(nulos_reviews)

                   user_id item_id                      posted recommend  \
137                  gdxsd     NaN                         NaN       NaN   
177      76561198094224872     NaN                         NaN       NaN   
2559     76561198021575394     NaN                         NaN       NaN   
3098             2ZESTY4ME     550            Posted March 11.      True   
4619     76561198093337643     550  Posted September 19, 2014.      True   
10080              cmuir37     NaN                         NaN       NaN   
13767            Jaysteeny     NaN                         NaN       NaN   
15493               ML8989     NaN                         NaN       NaN   
15981           terencemok  218620   Posted December 30, 2014.      True   
19184    76561198079215291     NaN                         NaN       NaN   
20223    76561198079342142     NaN                         NaN       NaN   
20486    76561197971285616  378041            Posted March 10.      True   
22057       

In [24]:
# Estos nulos estarían relacionados, serían usuarios que no tienen interacción en la plataforma, corresponden al tipo MNAR (Missing not at random). 
# Por tanto, eliminamos los 28 filas con nulos del dataframe 
df_reviews_final = df_reviews_final.dropna()

# Mostrar el dataframe actualizado
print(df_reviews_final)

                 user_id item_id                     posted recommend  \
0      76561197970982479    1250   Posted November 5, 2011.      True   
1      76561197970982479   22200      Posted July 15, 2011.      True   
2      76561197970982479   43110     Posted April 21, 2011.      True   
3                js41637  251610      Posted June 24, 2014.      True   
4                js41637  227300  Posted September 8, 2013.      True   
...                  ...     ...                        ...       ...   
59328  76561198312638244      70            Posted July 10.      True   
59329  76561198312638244  362890             Posted July 8.      True   
59330        LydiaMorley  273110             Posted July 3.      True   
59331        LydiaMorley     730            Posted July 20.      True   
59332        LydiaMorley     440             Posted July 2.      True   

                                                  review  
0      Simple yet with great replayability. In my opi...  
1    

Ajuste tipos de datos

In [25]:
# Finalmente se ajustan los tipos de datos para item_id y recommend

# Hacer una copia del df
df_reviews_final = df_reviews_final.copy()

# Convertir la columna 'item_id' a tipo entero, manejando nulos
df_reviews_final['item_id'] = pd.to_numeric(df_reviews_final['item_id'], errors='coerce').astype('int64')

# Convertir la columna 'recommend' a tipo booleano
df_reviews_final['recommend'] = df_reviews_final['recommend'].astype(bool)



In [26]:
print(df_reviews_final)

                 user_id  item_id                     posted  recommend  \
0      76561197970982479     1250   Posted November 5, 2011.       True   
1      76561197970982479    22200      Posted July 15, 2011.       True   
2      76561197970982479    43110     Posted April 21, 2011.       True   
3                js41637   251610      Posted June 24, 2014.       True   
4                js41637   227300  Posted September 8, 2013.       True   
...                  ...      ...                        ...        ...   
59328  76561198312638244       70            Posted July 10.       True   
59329  76561198312638244   362890             Posted July 8.       True   
59330        LydiaMorley   273110             Posted July 3.       True   
59331        LydiaMorley      730            Posted July 20.       True   
59332        LydiaMorley      440             Posted July 2.       True   

                                                  review  
0      Simple yet with great replayabili

In [27]:
df_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58401 entries, 0 to 59332
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    58401 non-null  object
 1   item_id    58401 non-null  int64 
 2   posted     58401 non-null  object
 3   recommend  58401 non-null  bool  
 4   review     58401 non-null  object
dtypes: bool(1), int64(1), object(3)
memory usage: 2.3+ MB


*Finalmente, corregimos la columna review para normalizar y eliminar todo lo que no sea texto*

In [28]:
col = df_reviews_final['review'].head(50)
print(col)

0     Simple yet with great replayability. In my opi...
1                  It's unique and worth a playthrough.
2     Great atmosphere. The gunplay can be a bit chu...
3     I know what you think when you see this title ...
4     For a simple (it's actually not all that simpl...
5     Very fun little game to play when your bored o...
6     A suitably punishing roguelike platformer.  Wi...
7     "Run for fun? What the hell kind of fun is that?"
8     Elegant integration of gameplay, story, world ...
9     Random drops and random quests, with stat poin...
10    Fun balance of tactics and strategy.  Potentia...
11    Fun world builder, with plenty of option of ho...
12    This game... is so fun. The fight sequences ha...
13    Really Really Really Great Game, very good sto...
14    Just buy it already. Great Story, Great Multip...
15    It was a great game from what I played, right ...
16    The ending to this game is.... ♥♥♥♥♥♥♥.... Jus...
17    Alan wake is a really good game, the light

In [29]:
# se define función para limpiar texto:
def clean_text(text):
    # Este codigo utiliza una expresión regular para eliminar todo lo que no sea texto y espacios en blanco
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    return cleaned_text

In [30]:
# Aplicar la función clean_text a la columna 'review'
df_reviews_final['review'] = df_reviews_final['review'].apply(clean_text)

In [31]:
# Verifico la limpieza realizada
columna = df_reviews_final['review'].head(50)
print(columna)

0     Simple yet with great replayability In my opin...
1                    Its unique and worth a playthrough
2     Great atmosphere The gunplay can be a bit chun...
3     I know what you think when you see this title ...
4     For a simple its actually not all that simple ...
5     Very fun little game to play when your bored o...
6     A suitably punishing roguelike platformer  Win...
7         Run for fun What the hell kind of fun is that
8     Elegant integration of gameplay story world de...
9     Random drops and random quests with stat point...
10    Fun balance of tactics and strategy  Potential...
11    Fun world builder with plenty of option of how...
12    This game is so fun The fight sequences have b...
13    Really Really Really Great Game very good stor...
14    Just buy it already Great Story Great Multipla...
15    It was a great game from what I played right n...
16    The ending to this game is  Just buy it youll ...
17    Alan wake is a really good game the light 

## 3. ETL de df_items

In [32]:
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


*Se crea copia para realizar las transformaciones*

In [33]:
df_items_copy = df_items.copy() # Hacemos una copia del DataFrame para hacer modificaciones del ETL

In [34]:
# Se aprecia fácilmente que la columna ítems está anidada. Necesitamos los datos de esta columna desanidados para cumplir con los objetivos del proyecto. Trabajaremos en desanidar estos datos y revisarlos.

# Revisamos los tipos de datos, estos están en los tipos correctos.

print(df_items_copy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB
None


In [35]:
# Se revisa primer valor de la columna items
primer_dato = df_items_copy['items'].iloc[0]
print(primer_dato)

[{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_2weeks': 0, 'playtime_forever': 6}
 {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_2weeks': 0, 'playtime_forever': 0}
 {'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_2weeks': 0, 'playtime_forever': 7}
 {'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_2weeks': 0, 'playtime_forever': 0}
 {'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_2weeks': 0, 'playtime_forever': 0}
 {'item_id': '60', 'item_name': 'Ricochet', 'playtime_2weeks': 0, 'playtime_forever': 0}
 {'item_id': '70', 'item_name': 'Half-Life', 'playtime_2weeks': 0, 'playtime_forever': 0}
 {'item_id': '130', 'item_name': 'Half-Life: Blue Shift', 'playtime_2weeks': 0, 'playtime_forever': 0}
 {'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_2weeks': 0, 'playtime_forever': 4733}
 {'item_id': '240', 'item_name': 'Counter-Strike: Source', 'playtime_2weeks': 0, 'playtime_forever': 1853}
 {'item_i

In [36]:
# Aplicar la función solo si es necesario
df_items_copy['items'] = df_items_copy['items'].apply(convert_to_list)

# Usar explode() para expandir los comentarios en varias filas
df_items_copy_expand = df_items_copy.explode('items').reset_index(drop=True)

# Desanidar el contenido de la columna 'reviews' en columnas separadas
df_items_final = pd.json_normalize(df_items_copy_expand['items'])

# Unir los datos originales con la información desanidada
df_items_final = pd.concat([df_items_copy_expand.drop(columns=['items']), df_items_final], axis=1)

In [37]:
print(df_items_final)

                   user_id  items_count           steam_id  \
0        76561197970982479          277  76561197970982479   
1        76561197970982479          277  76561197970982479   
2        76561197970982479          277  76561197970982479   
3        76561197970982479          277  76561197970982479   
4        76561197970982479          277  76561197970982479   
...                    ...          ...                ...   
5170010  76561198329548331            7  76561198329548331   
5170011  76561198329548331            7  76561198329548331   
5170012  76561198329548331            7  76561198329548331   
5170013  76561198329548331            7  76561198329548331   
5170014  edward_tremethick            0  76561198331598578   

                                                  user_url item_id  \
0        http://steamcommunity.com/profiles/76561197970...      10   
1        http://steamcommunity.com/profiles/76561197970...      20   
2        http://steamcommunity.com/profiles/7

In [38]:
print(df_items_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_2weeks   float64
 7   playtime_forever  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 315.6+ MB
None


In [39]:
# Contar el número de filas duplicadas
conteo_duplicados = df_items_final.duplicated().sum()

# Mostrar el resultado
print(f"Cantidad de filas duplicadas en df_items_final: {conteo_duplicados}")

Cantidad de filas duplicadas en df_items_final: 59196


In [40]:
# eliminamos las 59.104 filas duplicadas
df_items_final.drop_duplicates(inplace=True)
print(df_items_final)

                   user_id  items_count           steam_id  \
0        76561197970982479          277  76561197970982479   
1        76561197970982479          277  76561197970982479   
2        76561197970982479          277  76561197970982479   
3        76561197970982479          277  76561197970982479   
4        76561197970982479          277  76561197970982479   
...                    ...          ...                ...   
5170010  76561198329548331            7  76561198329548331   
5170011  76561198329548331            7  76561198329548331   
5170012  76561198329548331            7  76561198329548331   
5170013  76561198329548331            7  76561198329548331   
5170014  edward_tremethick            0  76561198331598578   

                                                  user_url item_id  \
0        http://steamcommunity.com/profiles/76561197970...      10   
1        http://steamcommunity.com/profiles/76561197970...      20   
2        http://steamcommunity.com/profiles/7

Tratamiento de nulos

In [41]:
# revisamos los nulos de este dataframe

print(df_items_final.isnull().sum())

user_id                 0
items_count             0
steam_id                0
user_url                0
item_id             16714
item_name           16714
playtime_2weeks     16714
playtime_forever    16714
dtype: int64


In [42]:
# Filtrar las filas donde hay valores nulos para las columnas 'item_id', 'item_name', 'playtime_forever'
nulos_items = df_items_final[df_items_final[['item_id', 'item_name', 'playtime_forever']].isna().any(axis=1)]

# Mostrar el resultado
nulos_items

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_2weeks,playtime_forever
3733,Wackky,0,76561198039117046,http://steamcommunity.com/id/Wackky,NaN,NaN,NaN,NaN
3849,76561198079601835,0,76561198079601835,http://steamcommunity.com/profiles/76561198079...,NaN,NaN,NaN,NaN
6019,hellom8o,0,76561198117222320,http://steamcommunity.com/id/hellom8o,NaN,NaN,NaN,NaN
6523,starkillershadow553,0,76561198059648579,http://steamcommunity.com/id/starkillershadow553,NaN,NaN,NaN,NaN
7237,darkenkane,0,76561198058876001,http://steamcommunity.com/id/darkenkane,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
5169470,76561198316380182,0,76561198316380182,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN,NaN
5169471,76561198316970597,0,76561198316970597,http://steamcommunity.com/profiles/76561198316...,NaN,NaN,NaN,NaN
5169472,76561198318100691,0,76561198318100691,http://steamcommunity.com/profiles/76561198318...,NaN,NaN,NaN,NaN
5170006,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,NaN,NaN,NaN,NaN


Estos valores faltantes serían del tipo MNAR (Missing not at random), ya que se trataría sólo de personas registradas en la plataforma sin interacción.  

Esto se intuye dado que si items_count es igual a 0, no pueden existir datos de item_id, item_name y playtime_forever.

In [43]:
# Verificamos esta hipotesis chequeando si items_count tiene un valor igual a 0 en todas las filas de nulos

conteo_nulos_items = nulos_items['items_count'].value_counts()
print(conteo_nulos_items)

items_count
0    16714
Name: count, dtype: int64


Como vemos, sólo existe el 0 como valor en la columna items_count, en este caso, serían usuarios sin interacción con la plataforma. Procedo a eliminar los nulos de df_items_final aplicando el método dropna()

In [44]:
# Eliminamos los nulos del dataframe 
df_items_final = df_items_final.dropna()

# Mostrar el dataframe actualizado
print(df_items_final.isnull().sum())

user_id             0
items_count         0
steam_id            0
user_url            0
item_id             0
item_name           0
playtime_2weeks     0
playtime_forever    0
dtype: int64


Ajuste tipos de datos

In [45]:
df_items_final.info()


<class 'pandas.core.frame.DataFrame'>
Index: 5094105 entries, 0 to 5170013
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_2weeks   float64
 7   playtime_forever  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 349.8+ MB


Todos los tipos de datos están correctos, excepto para item_id. Revisamos esta columna antes de proceder.

In [46]:
df_items_final['item_id'].sample(20)

2605183     91600
3026939    226720
3823232       620
3857652      9200
4351259    261640
2910306    239220
3705204    205230
3286303        70
2650993    224760
709437      31190
5056514    299820
3038806    105600
4241496    206440
317129      15130
4172391    204630
5166935    304050
4730198       730
601515      31280
4372924    221260
2469423    390880
Name: item_id, dtype: object

Los valores parecieran que son todos numéricos, por tanto realizamos el casteo.

In [47]:
# Hacer una copia del df
df_items_final = df_items_final.copy()

# Convertir la columna 'item_id' a tipo entero, manejando nulos
df_items_final['item_id'] = pd.to_numeric(df_items_final['item_id']).astype('int64')

In [48]:
# Verificamos el cambio
df_items_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094105 entries, 0 to 5170013
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           int64  
 5   item_name         object 
 6   playtime_2weeks   float64
 7   playtime_forever  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 349.8+ MB


Eliminación de columnas

In [49]:
# Eliminamos columnas que no necesitaremos: playtime_2weeks
col=['playtime_2weeks', 'user_url', 'steam_id']

df_items_final.drop(col,axis='columns', inplace=True)


In [50]:
# Visualizamos dataset
df_items_final

,user_id,items_count,item_id,item_name,playtime_forever
0,76561197970982479,277,10,Counter-Strike,6.0
1,76561197970982479,277,20,Team Fortress Classic,0.0
2,76561197970982479,277,30,Day of Defeat,7.0
3,76561197970982479,277,40,Deathmatch Classic,0.0
4,76561197970982479,277,50,Half-Life: Opposing Force,0.0
...,...,...,...,...,...
5170009,76561198329548331,7,346330,BrainBread 2,0.0
5170010,76561198329548331,7,373330,All Is Dust,0.0
5170011,76561198329548331,7,388490,One Way To Die: Steam Edition,3.0
5170012,76561198329548331,7,521570,You Have 10 Seconds 2,4.0


In [51]:
df_items_final.isnull().sum()

user_id             0
items_count         0
item_id             0
item_name           0
playtime_forever    0
dtype: int64

*Finalmente, limpiamos columna 'item_name' de caracteres especiales y símbolos*

In [52]:
def clean_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    return cleaned_text


# Aplicó la función a columna item_name
df_items_final['item_name'] = df_items_final['item_name'].apply(clean_text)


## 3. ETL de df_games

In [53]:
print(df_games)

              publisher                                 genres  \
0                  None                                   None   
1                  None                                   None   
2                  None                                   None   
3                  None                                   None   
4                  None                                   None   
...                 ...                                    ...   
120440  Ghost_RUS Games  [Casual, Indie, Simulation, Strategy]   
120441           Sacada              [Casual, Indie, Strategy]   
120442     Laush Studio            [Indie, Racing, Simulation]   
120443         SIXNAILS                        [Casual, Indie]   
120444             None                                   None   

                        app_name                     title  \
0                           None                      None   
1                           None                      None   
2                    

Se crea una copia del DF donde se realizarán todas las transformaciones.

In [54]:
df_games_clean = df_games.copy() # Hacemos una copia del DataFrame para hacer modificaciones del ETL

In [55]:
# Revisamos la cantidad de no- nulos
df_games_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


# Eliminación filas completamente Nulas


En esta parte, se descartarán las filas nulas que a simple vista podemos ver en este dataframe.

Primero hacemos una copia del DataFrame para hacer modificaciones del ETL

In [56]:
df_games_clean = df_games.copy() # Hacemos una copia del DataFrame para hacer modificaciones del ETL

In [57]:
# Definimos el número mínimo de valores no-None que una fila debe tener
thresh_value = 3  # Esto significa que las filas deben tener al menos 3 valores no-None para no ser eliminadas

# Eliminar filas que tienen más valores None que el umbral permitido
df_games_clean = df_games_clean.dropna(thresh=thresh_value)


In [58]:
df_games_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.4+ MB


In [59]:
# Mostrar el DataFrame limpio
df_games_clean.sample(5)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
110771,None,[Strategy],Star Realms - Heroes,Star Realms - Heroes,http://store.steampowered.com/app/499550/Star_...,2016-07-06,[Strategy],http://steamcommunity.com/app/499550/reviews/?...,"[Single-player, Multi-player, Cross-Platform M...",1.99,0.0,499550.0,White Wizard Games
111128,SecureMix LLC,[Utilities],GlassWire Basic,GlassWire Basic,http://store.steampowered.com/app/384530/Glass...,2016-06-07,[Utilities],http://steamcommunity.com/app/384530/reviews/?...,[Downloadable Content],49.0,0.0,384530.0,SecureMix LLC
97375,None,"[Casual, Simulation]",Rocksmith® 2014 Edition – Remastered – Avril L...,Rocksmith® 2014 Edition – Remastered – Avril L...,http://store.steampowered.com/app/509737/Rocks...,2017-03-07,"[Casual, Simulation]",http://steamcommunity.com/app/509737/reviews/?...,"[Single-player, Shared/Split Screen, Downloada...",2.99,0.0,509737.0,Ubisoft - San Francisco
99912,None,"[Adventure, Indie, Strategy]",Card City Nights 2 - Soundtrack,Card City Nights 2 - Soundtrack,http://store.steampowered.com/app/719710/Card_...,2017-09-21,"[Strategy, Adventure, Indie]",http://steamcommunity.com/app/719710/reviews/?...,"[Single-player, Online Multi-Player, Downloada...",3.99,0.0,719710.0,Ludosity
104213,"Pooandplay, Jogaloca","[Action, Casual, Indie]",Toys Gun Fire Boom,Toys Gun Fire Boom,http://store.steampowered.com/app/674900/Toys_...,2017-09-03,"[Action, Casual, Indie]",http://steamcommunity.com/app/674900/reviews/?...,"[Single-player, Multi-player, Co-op, Steam Ach...",5.99,0.0,674900.0,Pooandplay


# Tratamiento de valores faltantes

En esta sección analizaremos las posibles estrategias para llenar los campos faltantes en cada columna.

In [60]:
# Primero definimos una función para identificar valores faltantes

def missing_values_table(df):

    '''Función que devuelve los valores faltantes de cada columna de un dataset, y cuál es su porcentaje respecto del
    total de valores de cada columna'''
    
    miss_val = df.isnull().sum()
    miss_val_percent = 100 * df.isnull().sum() / len(df)
    
    # tabla con los resultados
    miss_val_table = pd.concat([miss_val, miss_val_percent], axis = 1)
    
    # Renombramos las columnas
    miss_val_table_ren_columns = miss_val_table.rename(columns = {0: 'Valores Faltantes', 1:
                                                               '% de Valores Totales'})
    # Ordenamos la tabla por el porcentaje en orden descendente
    miss_val_table_ren_columns = miss_val_table_ren_columns[
        miss_val_table_ren_columns.iloc[:,1] != 0].sort_values('% de Valores Totales',
                                                             ascending = False).round(1)
    
    # Imprimir el resumen de la información
    print('El dataframe seleccionado tiene '+ str(df.shape[1]) + ' columnas.\n'
         'Hay ' + str(miss_val_table_ren_columns.shape[0]) + 
         ' columnas que tienen valores faltantes.')
    
    # Devuelve el dataframe con informacion faltante
    return miss_val_table_ren_columns

In [61]:
# Usamos función recién creada
miss_df_games = missing_values_table(df_games_clean)
print('\nValores faltantes del dataset df_steam_clean:')
print(miss_df_games)

El dataframe seleccionado tiene 13 columnas.
Hay 11 columnas que tienen valores faltantes.

Valores faltantes del dataset df_steam_clean:
              Valores Faltantes  % de Valores Totales
publisher                  8052                  25.1
developer                  3299                  10.3
genres                     3283                  10.2
release_date               2067                   6.4
title                      2050                   6.4
price                      1377                   4.3
specs                       670                   2.1
tags                        163                   0.5
app_name                      2                   0.0
reviews_url                   2                   0.0
id                            2                   0.0


Hallazgos: 
- Las columnas app_name y title son iguales al parecer, pero se podrían rellenar algunos valores faltantes en alguna de estas columnas, y eliminar una.   
- Las columnas genre y tag comparten información sobre el género del juego. Se completará genre, que es una columna que necesitamos, con los valores de tags (que posee sólo 163 nulos), y se eliminará tags.
- Se deberá buscar una estrategia para el tratamiento de los nombres de ítem escritos en alfabeto distinto al latino.
- Los valores faltantes en columnas clave son altos: publisher, developer, genres. Se evaluará con mayor información si eliminamos algunas columnas luego de hacer algunas transformaciones.
- Algunos nombres de publisher coinciden con developer, pero en la mayor parte de los casos no lo hacen.
- url contiene el nombre (title) del juego, se puede usar para completar la columna title si es necesraio.
- Columnas candidatas a eliminar: app_name o title (fusionar), url, tags, reviews_url, specs y early_access. 

In [62]:
# revisamos columnas app_name y title, qué tantos valores coinciden
son_iguales = df_games_clean['app_name'].equals(df_games_clean['title'])
conteo_iguales = (df_games_clean['app_name'] == df_games_clean['title']).sum()

# Mostrar el resultado
print(f"Cantidad de filas donde 'app_name' y 'title' son iguales: {conteo_iguales}")

# Contar cuántos valores son diferentes
conteo_diferentes = (df_games_clean['app_name'] != df_games_clean['title']).sum()
print(f"Cantidad de filas donde 'app_name' y 'title' son diferentes: {conteo_diferentes}")


if not son_iguales:
    diferencias = df_games_clean[df_games_clean['app_name'] != df_games_clean['title']]
    
print(f'Conteo nulos:', diferencias.isnull().sum())



Cantidad de filas donde 'app_name' y 'title' son iguales: 29530
Cantidad de filas donde 'app_name' y 'title' son diferentes: 2605
Conteo nulos: publisher       2231
genres          2063
app_name           2
title           2050
url                0
release_date    2049
tags               3
reviews_url        1
specs             19
price            127
early_access       0
id                 1
developer       2060
dtype: int64


La gran parte de las diferencias entre app_name y title corresponde a valores nulos que posee title. 'App_name posee un solo nulo, nos quedamos con esta columna y eliminamos title. 

In [63]:
# Filtrar las filas donde 'title' no sea nulo y sea diferente de 'app_name'
diferentes = df_games_clean[(df_games_clean['title'].notna()) & (df_games_clean['title'] != df_games_clean['app_name'])]

# Mostrar el resultado
diferentes[['app_name', 'title']]

,app_name,title
88390,Sam & Max 101: Culture Shock,Sam &amp; Max 101: Culture Shock
88393,Sam & Max 102: Situation: Comedy,Sam &amp; Max 102: Situation: Comedy
88419,Command & Conquer: Red Alert 3,Command &amp; Conquer: Red Alert 3
88492,Heroes of Might & Magic V: Hammers of Fate,Heroes of Might &amp; Magic V: Hammers of Fate
88494,Heroes of Might & Magic V: Tribes of the East,Heroes of Might &amp; Magic V: Tribes of the East
...,...,...
120181,Sam & Max 105: Reality 2.0,Sam &amp; Max 105: Reality 2.0
120182,Sam & Max 104: Abe Lincoln Must Die!,Sam &amp; Max 104: Abe Lincoln Must Die!
120183,Sam & Max 106: Bright Side of the Moon,Sam &amp; Max 106: Bright Side of the Moon
120208,Making History: The Calm & the Storm,Making History: The Calm &amp; the Storm


Las diferencias al parecer sería porque en title no reconoce el ampersand. Lo corregimos y evaluamos.

In [64]:
# Reemplazar '&amp;' por '&' en la columna 'title'
df_games_clean['title'] = df_games_clean['title'].str.replace('&amp;', '&', regex=False)

# Mostrar los primeros registros para verificar el cambio
print(df_games_clean[['app_name', 'title']].head())

                      app_name                    title
88310      Lost Summoner Kitty      Lost Summoner Kitty
88311                Ironbound                Ironbound
88312  Real Pool 3D - Poolians  Real Pool 3D - Poolians
88313                  弹炸人2222                  弹炸人2222
88314            Log Challenge                     None


Evaluamos el conteo de nulos nuevamente

In [65]:

# revisamos columnas app_name y title, qué tantos valores coinciden
son_iguales = df_games_clean['app_name'].equals(df_games_clean['title'])
conteo_iguales = (df_games_clean['app_name'] == df_games_clean['title']).sum()

# Mostrar el resultado
print(f"Cantidad de filas donde 'app_name' y 'title' son iguales: {conteo_iguales}")

# Contar cuántos valores son diferentes
conteo_diferentes = (df_games_clean['app_name'] != df_games_clean['title']).sum()
print(f"Cantidad de filas donde 'app_name' y 'title' son diferentes: {conteo_diferentes}")


if not son_iguales:
    diferencias = df_games_clean[df_games_clean['app_name'] != df_games_clean['title']]
    
print(f'Conteo nulos:', diferencias.isnull().sum())

Cantidad de filas donde 'app_name' y 'title' son iguales: 30080
Cantidad de filas donde 'app_name' y 'title' son diferentes: 2055
Conteo nulos: publisher       2050
genres          2049
app_name           2
title           2050
url                0
release_date    2049
tags               2
reviews_url        1
specs              1
price            117
early_access       0
id                 1
developer       2050
dtype: int64


In [66]:
# Filtrar nuevamente las filas donde 'title' no sea nulo y sea diferente de 'app_name'
diferentes2 = df_games_clean[(df_games_clean['title'].notna()) & (df_games_clean['title'] != df_games_clean['app_name'])]

# Mostrar el resultado
diferentes2[['app_name', 'title']]

,app_name,title
88541,Guild Wars Nightfall,Guild Wars Nightfall®
88542,Guild Wars,Guild Wars® Game of the Year Edition
88543,Guild Wars Factions,Guild Wars Factions®
99323,>observer_,&gt;observer_
106020,>//:System.Hack,&gt;//:System.Hack


El resultado anterior nos muestra las diferencias que aún persisten entre app_name y title, debido a caracteres especiales. Podemos concluir que title y app_name son prácticamente iguales. Nos quedamos con app_name y eliminamos title.

In [67]:
# Eliminar la columna 'title' , además de las columnas 'url', 'user_url', 'early_access', que no necesitaremos
col = ['title', 'url', 'specs', 'reviews_url', 'early_access']
df_games_clean = df_games_clean.drop(columns=col)

In [68]:
# Mostrar el DataFrame actualizado
print(df_games_clean)

               publisher                                             genres  \
88310          Kotoshiro      [Action, Casual, Indie, Simulation, Strategy]   
88311   Making Fun, Inc.               [Free to Play, Indie, RPG, Strategy]   
88312       Poolians.com  [Casual, Free to Play, Indie, Simulation, Sports]   
88313               彼岸领域                        [Action, Adventure, Casual]   
88314               None                                               None   
...                  ...                                                ...   
120440   Ghost_RUS Games              [Casual, Indie, Simulation, Strategy]   
120441            Sacada                          [Casual, Indie, Strategy]   
120442      Laush Studio                        [Indie, Racing, Simulation]   
120443          SIXNAILS                                    [Casual, Indie]   
120444              None                                               None   

                        app_name release_date  \
88

Análisis columna publisher

In [69]:
# Se filtran los valores nulos de la columna publisher para observar

filtro_publisher= df_games_clean[df_games_clean['publisher'].isna()]

In [70]:
# Se selecciona una muestra para revisar patrones en los datos
filtro_publisher.sample(10)

,publisher,genres,app_name,release_date,tags,price,id,developer
114648,None,"[Adventure, Casual, Indie, RPG, Simulation]",FREE Soccer Theme Pack,2015-06-17,"[Adventure, RPG, Indie, Casual, Simulation]",None,380760.0,Stolen Couch Games
97185,None,None,Andy Kaufman: My Breakfast With Blassie,1983-11-01,[Movie],2.99,596010.0,None
95924,None,"[Action, Casual, Indie]",Project Starship OST,2016-10-18,"[Action, Indie, Casual]",0.99,541830.0,xXarabonXx
90810,None,"[Casual, Simulation]",Rocksmith® 2014 – Biffy Clyro - “Mountains”,2014-08-05,"[Casual, Simulation]",2.99,294975.0,Ubisoft - San Francisco
117232,None,"[Casual, Simulation]",Rocksmith® 2014 – Sum 41 - “In Too Deep”,2014-06-03,"[Casual, Simulation]",2.99,294925.0,Ubisoft - San Francisco
96005,None,None,Island Queen,2012-05-04,"[Short, Comedy, Drama]",1.99,413250.0,None
94198,None,None,Saw 3,2007-01-23,"[Movie, Mystery, Crime, Horror, Thriller]",3.99,421450.0,None
114450,None,"[Action, Casual, Indie]",TeraBlaster,2015-07-10,"[Indie, Casual, Action, Retro, Shoot 'Em Up]",Free to Play,384150.0,Black Shell Games
106393,None,None,Drone Fighters,None,"[Early Access, Action, Indie, Simulation, Spor...",6.99,612600.0,None
110059,None,None,Piñata,None,"[Free to Play, Indie, Casual, Simulation, VR]",Free To Play,477800.0,None


Revisar en qué coinciden publisher y developer


In [71]:
# Contar cuántas veces ambas columnas son nulas
ambos_nulos = df_games_clean[df_games_clean['publisher'].isnull() & df_games_clean['developer'].isnull()].shape[0]
print(f"Cantidad de filas donde 'publisher' y 'developer' son nulos: {ambos_nulos}")

# Contar cuántas veces sólo 'publisher' es nulo y 'developer' no lo es
publisher_nulo = df_games_clean[df_games_clean['publisher'].isnull() & df_games_clean['developer'].notnull()].shape[0]
print(f"Cantidad de filas donde 'publisher' es nulo y 'developer' no es nulo: {publisher_nulo}")

# Contar cuántas veces sólo 'developer' es nulo y 'publisher' no lo es
developer_nulo = df_games_clean[df_games_clean['developer'].isnull() & df_games_clean['publisher'].notnull()].shape[0]
print(f"Cantidad de filas donde 'developer' es nulo y 'publisher' no es nulo: {developer_nulo}")

coinciden_no_nulos = df_games_clean[(df_games_clean['publisher'] == df_games_clean['developer']) & df_games_clean['publisher'].notnull()].shape[0]
print(f"Cantidad de filas donde 'publisher' y 'developer' son iguales (excluyendo nulos): {coinciden_no_nulos}")

diferentes_no_nulos = df_games_clean[(df_games_clean['publisher'] != df_games_clean['developer']) & df_games_clean['publisher'].notnull() & df_games_clean['developer'].notnull()].shape[0]
print(f"Cantidad de filas donde 'publisher' y 'developer' son diferentes y ambos no son nulos: {diferentes_no_nulos}")

Cantidad de filas donde 'publisher' y 'developer' son nulos: 3234
Cantidad de filas donde 'publisher' es nulo y 'developer' no es nulo: 4818
Cantidad de filas donde 'developer' es nulo y 'publisher' no es nulo: 65
Cantidad de filas donde 'publisher' y 'developer' son iguales (excluyendo nulos): 12432
Cantidad de filas donde 'publisher' y 'developer' son diferentes y ambos no son nulos: 11586


In [72]:
# Filtrar las filas donde tanto 'publisher' como 'developer' son nulos
filtro_nulos = df_games_clean[(df_games_clean['publisher'].isnull()) & (df_games_clean['developer'].isnull())]

filtro_nulos.head()

,publisher,genres,app_name,release_date,tags,price,id,developer
88314,None,None,Log Challenge,None,"[Action, Indie, Casual, Sports]",2.99,773570.0,None
88321,None,None,Icarus Six Sixty Six,None,[Casual],Free,724910.0,None
88329,None,None,After Life VR,None,"[Early Access, Indie, VR]",4.99,772590.0,None
88330,None,None,Kitty Hawk,None,"[Early Access, Action, Adventure, Indie, Casual]",2.99,640250.0,None
88332,None,None,Mortars VR,None,"[Early Access, Strategy, Action, Indie, Casual...",0.99,711440.0,None


*Excluyendo a los valores nulos, existen 12.432 datos que son iguales entre publisher y developer. Nos centramos en revisar el último resultado, 'Cantidad de filas donde 'publisher' y 'developer' son diferentes y ambos no son nulos: 11586' para ver qué tanto difieren estos datos entre si.*

In [73]:
# Filtrar las filas donde 'publisher' y 'developer' son distintos y ambos no son nulos
diferentes_no_nulos = df_games_clean[(df_games_clean['publisher'] != df_games_clean['developer']) & df_games_clean['publisher'].notnull() & df_games_clean['developer'].notnull()]


In [74]:
# Mostrar el DataFrame filtrado
diferentes_no_nulos[['publisher', 'developer']].sample(10)

,publisher,developer
113877,Dagestan Technology,Hipix Studio
102599,ArcaneRaise,Arcane Raise
97605,NewRealityGames,Lapovich
106983,Degica,NIVLACART
108789,Big Fish Games,Elephant Games
102026,Refproto Games,Mianwotu
118241,2K Games,2K Marin
99693,Laush Studio,Laush Dmitriy Sergeevich
108499,rokapublish,bumblebee
91371,505 Games,"Lion Game Lion,OVERKILL - a Starbreeze Studio."


Al revisar las sucesivas muestras de datos, estos 11.586 datos difieren en casi su totalidad, por tanto no podemos generalizar e imputar los datos de developer a publisher. No eliminaremos estas filas dado que el resto de columnas tiene información relevante para nuestro proyecto. Por ahora decidimos dejarlos así.

Análisis columnas tags y genres

En el análisis preliminar, se pudo advertir que estas columnas comparten información respecto del género del videojuego. Aún así, hay valores que no corresponden al género de videojuego, como 'Free to Play' y 'Early Acces', que eliminaremos.

In [75]:
# Revisamos los valores de 
df_games_clean.isnull().sum()

publisher       8052
genres          3283
app_name           2
release_date    2067
tags             163
price           1377
id                 2
developer       3299
dtype: int64

In [76]:
# Revisamos todos los géneros 
all_genres=df_games_clean['genres'].explode().unique().tolist()
all_genres

['Action',
 'Casual',
 'Indie',
 'Simulation',
 'Strategy',
 'Free to Play',
 'RPG',
 'Sports',
 'Adventure',
 None,
 'Racing',
 'Early Access',
 'Massively Multiplayer',
 'Animation &amp; Modeling',
 'Video Production',
 'Utilities',
 'Web Publishing',
 'Education',
 'Software Training',
 'Design &amp; Illustration',
 'Audio Production',
 'Photo Editing',
 'Accounting']

In [77]:
# Revisamos el contenido de los tags, en busca de palabras clave asociadas al género del video juego.
all_tags=df_games_clean['tags'].explode().unique().tolist()
len(all_tags)


340

*Hay 340 tipos de 'tags' en la columna tags. La estrategia a utilizar será completar los valores faltantes de genres con la columna tags.*

In [78]:
# Se actualiza la columna genres con los valores de tags:
df_games_clean.loc[:, 'genres'] = df_games_clean['genres'].fillna(df_games_clean['tags'])

En columna 'tag' también podemos encontrar el género de la película en muchos casos, por tanto, se podría utilizar alguna estrategia para rellenar los datos faltantes en 'genre'. Por otro lado, se identifican algunos géneros listados que no lo son: 'Free to Play', 'Early Acces', que se eliminarán de 'genre'.

In [79]:
df_games_clean.isnull().sum()

publisher       8052
genres           139
app_name           2
release_date    2067
tags             163
price           1377
id                 2
developer       3299
dtype: int64

In [80]:
all_genres=df_games_clean['genres'].explode().unique().tolist()
print(len(all_genres))

304


*Como vemos, nuestro listado de géneros aumentó a 304, necesitamos simplificar y reducir este listado más adelante al seleccionar un sólo tipo de género listado para cada juego. Hacemos algunas modificaciones en este nuevo listado.*

In [81]:

df_games_clean['genres'] = df_games_clean['genres'].replace('Politics', 'Political')
# Reemplazar '&amp;' por '&' en la columna 'genres'
df_games_clean['genres'] = df_games_clean['genres'].astype(str).str.replace('&amp;', '&', regex=False)


In [82]:
# Convertir los valores nulos a cadenas vacías para evitar errores
df_games_clean['genres'] = df_games_clean['genres'].fillna('')

# Eliminar los términos específicos de la columna 'genres'
df_games_clean['genres'] = df_games_clean['genres'].str.replace(
    r'\b(Free to Play|Early Access|First-Person|Singleplayer|Difficult|Great Soundtrack|Multiplayer|Local Multiplayer|Asynchronous Multiplayer|4 Player Local|Soundtrack|Funny|Software)\b,?\s?', 
    '', 
    regex=True
)

In [83]:
all_genres_ev=df_games_clean['genres'].explode().unique().tolist()

In [84]:
all_genres_ev

["['Action', 'Casual', 'Indie', 'Simulation', 'Strategy']",
 "['', 'Indie', 'RPG', 'Strategy']",
 "['Casual', '', 'Indie', 'Simulation', 'Sports']",
 "['Action', 'Adventure', 'Casual']",
 "['Action', 'Indie', 'Casual', 'Sports']",
 "['Action', 'Adventure', 'Simulation']",
 "['', 'Indie', 'Simulation', 'Sports']",
 "['Casual', 'Indie', 'Racing', 'Simulation']",
 "['Action', 'Indie', 'Simulation', '']",
 "['Casual']",
 "['Action', 'Adventure', 'Casual', 'Indie', 'RPG']",
 "['Casual', 'Indie']",
 "['Casual', 'Indie', 'Simulation']",
 "['Adventure', 'Casual', 'Indie', 'Simulation', 'Strategy']",
 "['', 'Indie', 'VR']",
 "['', 'Action', 'Adventure', 'Indie', 'Casual']",
 "['Action', 'Adventure', 'Indie']",
 "['', 'Strategy', 'Action', 'Indie', 'Casual', 'VR']",
 "['Racing', 'Simulation', 'Sports']",
 "['Action', 'Indie']",
 "['Design & Illustration', 'Tutorial']",
 "['Action', 'Indie', 'Racing']",
 "['Action']",
 "['Action', 'Indie', 'RPG']",
 "['Action', 'Indie', 'VR']",
 "['Casual', 'Indi

In [85]:
# Convertir los datos de la columna 'tags' a cadenas
df_games_clean[['genres', 'tags', 'publisher', 'app_name', 'developer']] = df_games_clean[['genres', 'tags', 'publisher', 'app_name', 'developer']].astype(str)

In [86]:
# Limpiamos los textos de nuestro dataframe. Creamos función para convertir listas a cadenas y limpiar el texto
def clean_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    return cleaned_text
# Aplicar la función a las columnas especificadas
#['genres', 'tags', 'specs', 'publisher', 'app_name', 'developer']
columns_to_clean = ['genres', 'tags', 'publisher', 'app_name', 'developer']
for column in columns_to_clean:
    df_games_clean[column] = df_games_clean[column].apply(clean_text)


In [87]:
df_games_clean.head()

,publisher,genres,app_name,release_date,tags,price,id,developer
88310,Kotoshiro,Action Casual Indie Simulation Strategy,Lost Summoner Kitty,2018-01-04,Strategy Action Indie Casual Simulation,4.99,761140.0,Kotoshiro
88311,Making Fun Inc,Indie RPG Strategy,Ironbound,2018-01-04,Free to Play Strategy Indie RPG Card Game Trad...,Free To Play,643980.0,Secret Level SRL
88312,Poolianscom,Casual Indie Simulation Sports,Real Pool D Poolians,2017-07-24,Free to Play Simulation Sports Casual Indie Mu...,Free to Play,670290.0,Poolianscom
88313,,Action Adventure Casual,,2017-12-07,Action Adventure Casual,0.99,767400.0,
88314,None,Action Indie Casual Sports,Log Challenge,None,Action Indie Casual Sports,2.99,773570.0,None


In [88]:
# Transformar los "Free to play" en "Free"
df_games_clean['price'] = df_games_clean['price'].replace('(?i)Free to Play', 'Free', regex=True)

In [89]:
# Actualizar la columna 'price' a 'Free' donde 'tags' contiene 'Free to Play'
df_games_clean.loc[df_games_clean['tags'].str.contains('Free to Play', na=False), 'price'] = 'Free'

In [90]:
df_games_clean.head()

,publisher,genres,app_name,release_date,tags,price,id,developer
88310,Kotoshiro,Action Casual Indie Simulation Strategy,Lost Summoner Kitty,2018-01-04,Strategy Action Indie Casual Simulation,4.99,761140.0,Kotoshiro
88311,Making Fun Inc,Indie RPG Strategy,Ironbound,2018-01-04,Free to Play Strategy Indie RPG Card Game Trad...,Free,643980.0,Secret Level SRL
88312,Poolianscom,Casual Indie Simulation Sports,Real Pool D Poolians,2017-07-24,Free to Play Simulation Sports Casual Indie Mu...,Free,670290.0,Poolianscom
88313,,Action Adventure Casual,,2017-12-07,Action Adventure Casual,0.99,767400.0,
88314,None,Action Indie Casual Sports,Log Challenge,None,Action Indie Casual Sports,2.99,773570.0,None


In [91]:
# Reemplazar todos los valores vacíos ('') en el df_reviews_final por NaN
df_games_clean.replace('', np.nan, inplace=True)
df_games_clean.replace('None', np.nan, inplace=True)

In [92]:
df_games_clean.head()

,publisher,genres,app_name,release_date,tags,price,id,developer
88310,Kotoshiro,Action Casual Indie Simulation Strategy,Lost Summoner Kitty,2018-01-04,Strategy Action Indie Casual Simulation,4.99,761140.0,Kotoshiro
88311,Making Fun Inc,Indie RPG Strategy,Ironbound,2018-01-04,Free to Play Strategy Indie RPG Card Game Trad...,Free,643980.0,Secret Level SRL
88312,Poolianscom,Casual Indie Simulation Sports,Real Pool D Poolians,2017-07-24,Free to Play Simulation Sports Casual Indie Mu...,Free,670290.0,Poolianscom
88313,NaN,Action Adventure Casual,NaN,2017-12-07,Action Adventure Casual,0.99,767400.0,NaN
88314,NaN,Action Indie Casual Sports,Log Challenge,None,Action Indie Casual Sports,2.99,773570.0,NaN


In [93]:
# revisamos nuevamente los missing values

# Usamos función missing_values_table que ya creamos anteriormente
miss_df_games_clean = missing_values_table(df_games_clean)
print('\nValores faltantes del dataset df_games_clean:')
print(miss_df_games_clean)

El dataframe seleccionado tiene 8 columnas.
Hay 8 columnas que tienen valores faltantes.

Valores faltantes del dataset df_games_clean:
              Valores Faltantes  % de Valores Totales
publisher                  8100                  25.2
developer                  3351                  10.4
release_date               2067                   6.4
price                      1173                   3.7
genres                      167                   0.5
tags                        163                   0.5
app_name                     26                   0.1
id                            2                   0.0


*Dado que los nulos son significativos en el caso de publisher. developer y release_data, en el EDA buscaremos estartegias para tratarlos, ya que eliminarlos desde ya eliminaría gran parte de la información de nuestro dataframe.*

*Sólo se deja año en la columna release_date de la nueva base de datos y se ajusta el tipo de dato precio a float*

In [94]:
# Filtrar filas donde 'price' sea 'Free' o nulo
nulos_price = df_games_clean[df_games_clean['price'].isnull()]



In [95]:
# Mostrar el resultado
nulos_price [['tags', 'price']].sample(10)

,tags,price
93707,Design Illustration Web Publishing,None
116492,Adventure,None
118734,Action RPG,None
108784,Action Indie Casual VR,None
109561,Action Indie Sports,None
114274,NaN,None
102702,Adventure Indie,None
95047,Action Indie VR Flight,None
101259,Casual Adventure Visual Novel Anime Sexual Con...,None
104055,Action Indie Casual,None


In [96]:
df_games_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24035 non-null  object 
 1   genres        31968 non-null  object 
 2   app_name      32109 non-null  object 
 3   release_date  30068 non-null  object 
 4   tags          31972 non-null  object 
 5   price         30962 non-null  object 
 6   id            32133 non-null  float64
 7   developer     28784 non-null  object 
dtypes: float64(1), object(7)
memory usage: 2.2+ MB


*Normalizamos la columa price*

In [97]:

# Se define una función lambda para convertir a 0 si es string, de lo contrario mantener el valor original
replace_string_with_zero = lambda x: 0 if isinstance(x, str) else x

# Aplicar la función lambda a la columna 'price'
df_games_clean['price'] = df_games_clean['price'].apply(replace_string_with_zero)

# Verificar los cambios
print(df_games_clean['price'])

88310     4.99
88311     0.00
88312     0.00
88313     0.99
88314     2.99
          ... 
120440    1.99
120441    4.99
120442    1.99
120443    4.99
120444    4.99
Name: price, Length: 32135, dtype: float64


*En columna release_data, sólo dejamos el año:*

In [98]:
df_games_clean=df_games_clean.copy()
# Normalizar la columna 'release_date'
df_games_clean['release_date'] = df_games_clean['release_date'].str.extract(r'(\d{4})')

df_games_clean

,publisher,genres,app_name,release_date,tags,price,id,developer
88310,Kotoshiro,Action Casual Indie Simulation Strategy,Lost Summoner Kitty,2018,Strategy Action Indie Casual Simulation,4.99,761140.0,Kotoshiro
88311,Making Fun Inc,Indie RPG Strategy,Ironbound,2018,Free to Play Strategy Indie RPG Card Game Trad...,0.00,643980.0,Secret Level SRL
88312,Poolianscom,Casual Indie Simulation Sports,Real Pool D Poolians,2017,Free to Play Simulation Sports Casual Indie Mu...,0.00,670290.0,Poolianscom
88313,NaN,Action Adventure Casual,NaN,2017,Action Adventure Casual,0.99,767400.0,NaN
88314,NaN,Action Indie Casual Sports,Log Challenge,NaN,Action Indie Casual Sports,2.99,773570.0,NaN
...,...,...,...,...,...,...,...,...
120440,GhostRUS Games,Casual Indie Simulation Strategy,Colony On Mars,2018,Strategy Indie Casual Simulation,1.99,773640.0,Nikita GhostRUS
120441,Sacada,Casual Indie Strategy,LOGistICAL South Africa,2018,Strategy Indie Casual,4.99,733530.0,Sacada
120442,Laush Studio,Indie Racing Simulation,Russian Roads,2018,Indie Simulation Racing,1.99,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,Casual Indie,EXIT Directions,2017,Indie Casual Puzzle Singleplayer Atmospheric R...,4.99,658870.0,xropistevns


In [99]:
df_games_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24035 non-null  object 
 1   genres        31968 non-null  object 
 2   app_name      32109 non-null  object 
 3   release_date  29967 non-null  object 
 4   tags          31972 non-null  object 
 5   price         30962 non-null  float64
 6   id            32133 non-null  float64
 7   developer     28784 non-null  object 
dtypes: float64(2), object(6)
memory usage: 2.2+ MB


*Guardamos finalmente los tres dataframes en formato parquet*

In [100]:
df_games_clean.to_parquet('Steam_games.parquet', index=False)
df_reviews_final.to_parquet('Reviews.parquet', index=False)
df_items_final.to_parquet('Items.parquet', index=False)